In [ ]:

%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd 
import numpy as np

import torch 
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from modules.classifer_utils import ClassifierDataset, TrainingManager

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")


In [ ]:

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
heart_csv_df = pd.read_csv(file_url)

heart_csv_df

In [ ]:
## TODO consider skip last!


[encoded_df] = ClassifierDataset.onehot_encode_datafames( [heart_csv_df] )

overall_ds = ClassifierDataset(encoded_df, "target")

train_ds, test_ds = random_split(overall_ds, [.75, .25])

batch_size = int(len(train_ds) / 10)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

print(f'{len(train_ds)} training records in with batch size {batch_size}, {len(test_ds)} records for test')

first_training_record, _ = train_ds[0]
num_features = first_training_record.shape[-1]
print(f'datasets have {num_features} features')


In [ ]:
DROPOUT_RATE_01 = .2

class HeartRateNN(nn.Module):
    def __init__(self, num_feature_columns):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(num_feature_columns, 32),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.GELU(),
            nn.Linear(32, 16),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.GELU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear_stack(x)
    

first_training_record, _ = train_ds[0]
num_features = first_training_record.shape[-1]

model = HeartRateNN(num_features)
print(model)

In [ ]:

first_training_record, _ = train_ds[0]
num_feature_columns = first_training_record.shape[-1]

model = HeartRateNN(num_feature_columns)

training_mgr = TrainingManager(model)
training_mgr.train(train_dataloader, 100)


In [ ]:

training_mgr.eval(test_dataloader)
